In [ ]:
% load_ext autoreload
% autoreload 2

%matplotlib inline

In [ ]:
from dataset import *
from model import *
from solver import *

In [ ]:
#data_train = moving_mnist_dataset(os.path.join("..", "data", "mnist", "mnist_test_seq.npy"))
anno_path = os.path.join("..", "data", "train", "Annotations", "train.csv")
img_folder = os.path.join("..", "data", "train")
data_train = fashion_ai_dataset(img_folder, anno_path)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

from random import randint
from torchvision.transforms import ToPILImage, Normalize

def show_data(x, y, y_pred=None):
    to_img = ToPILImage()
    
    def kpts_img(kpts, blur=True):
        img = torch.sum(kpts, 0, keepdim=True)
        img = to_img(img)
        img = np.array(img)
        if blur:
            img = cv2.GaussianBlur(img, (5, 5), 1)
        return img
    
    num_img = 2 if y_pred is None else 3
    plt.figure(figsize=(18, 18 // num_img))
    
    # undo the normalize by resnet
    mu = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    mu = -mu / std
    std = 1 / std
    normalize = Normalize(mu, std)
    plt.subplot(1, num_img, 1)
    plt.imshow(to_img(normalize(x)))
    
    mask, y = y
    y *= mask
    plt.subplot(1, num_img, 2)
    plt.imshow(kpts_img(y, blur=False))
    
    if y_pred is not None:
        y_pred *= mask
        plt.subplot(1, num_img, 3)
        plt.imshow(kpts_img(y_pred))

In [ ]:
idx = randint(0, len(data_train) - 1)
#idx = 19629
print(data_train.dataset.df.iloc[idx])
show_data(*data_train[idx])

In [ ]:
import torch
import torch.nn as nn

model = CascadePyramidNet(24)
#model.load_state_dict(torch.load("w.h5"))
loss_fn = LossWithMask(nn.MSELoss())
optim = torch.optim.Adam(model.parameters(), lr=0.001)

solver = Solver(model, optim, loss_fn, loss_fn)

In [ ]:
solver.fit(data_train, 1, batch_size=16, print_cnt=50)

In [ ]:
torch.save(model.state_dict(), "w.h5")

In [ ]:
idx = randint(0, len(data_train) - 1)
#idx = 19629
print(data_train.dataset.df.iloc[idx])

x, y = data_train[idx]
show_data(x, y, model(x.unsqueeze(0).cuda()).squeeze(0).cpu())